    Done on Jupyter notebook separately.
    It took lot of time to read and preprocess the data in my low specification system.
    The functions are applied on small chunks of data and the results were combined to get the final output.
    After creating the final corpus, I shifted to Colab.
    To read and preprocess the data in my low specification was the toughest job. Most of the time the kernel became dead or I got memoryerror.
    That is why I decided to do it in chunks and then combine the result.
    It can be done on colab but as I spent most of my time writing and running this code I decided to share this file.

### Write the Corpus in a text file

In [0]:
from wiki_dump_reader import Cleaner, iterate

def write_corpus():
    corpus_file = '/home/rohan/CMI/SEM_3/Text_analytics/Bengali_corpus.txt'
    page_count = 0
    cleaner = Cleaner()
    with open(corpus_file, 'w', encoding='utf-8') as output:
        for title, text in iterate('/home/rohan/CMI/SEM_3/Text_analytics/bnwiki-latest-pages-articles.xml'):
            text = cleaner.clean_text(text)
            cleaned_text, links = cleaner.build_links(text)
            output.write(title + '\n' + cleaned_text + '\n')
            page_count += 1
            if page_count % 1000 == 0:
                print('Pages dumped = ', page_count)

    output.close()
write_corpus()

### Read the Corpus from the text file

In [0]:
import time
start=time.time()
file = open('/home/rohan/CMI/SEM_3/Text_analytics/Bengali_corpus.txt','r')
data = file.read()
end=time.time()
print(end-start)

14.090218544006348


### Preprocessing

#### Function to remove the special characters

In [0]:
import re
special_characters="=-/,!@|#$&:;$"
def remove_special_characters(text):
    for i in range(len(special_characters)):
        text =  re.sub(str(special_characters[i]),"", text)
    text =  re.sub("\n"," ", text)
    return(text)   

#### A function to remove any non-bengali words

In [0]:
def check_bengali_word(word): 
    try:        
        lang = unicodedata.name(word.strip()[0])        
        if 'BENGALI' in lang and 'DIGIT' not in lang:            
            return True        
        else:            
            return False    
    except:        
        return False 

In [0]:
len(data)

244751468

##### As I have very low specification system, operations on the whole data lead to memoryerror or dead kernel. So to avoid that and to be able to work in my own system, I decided to apply the functions on small chunks of data and combining the results

#### Removing the special characters

In [0]:
##It is done by reading the data in chunks instead of reading the whole data

start=time.time()
window_size=100000
mod_data=""
i=0
while i<len(data):
        mod_data+=remove_special_characters(data[i:i+100000])
        i=i+100000
        if i%50000000==0:
            print("Proceeding, Time:",time.time())  ## To check the procedding
print(i)
## For the last chunk of data
mod_data+=remove_special_characters(data[i-100000:])
end=time.time()
print("Time taken:",end-start)

Proceeding, Time: 1573368913.857451
Proceeding, Time: 1573368939.6548204
Proceeding, Time: 1573368964.2493842
Proceeding, Time: 1573368989.1016688
244800000
Time taken: 122.74754285812378


In [0]:
len(mod_data)

239849139

#### A function to find space from a text nearest to any given index

In [0]:
def find_nearst_index_for_space(text,index):
    window=25
    for i in range(window):
        if text[index-i]==" ":
            return index-i
        elif text[index+i]==" ":
            return index+i
        else:
            pass

#### Removing the non-bengali words

#### Target is to apply the function on small chunks of data

In [0]:
modified_corpus=""
i=k=0
while i<len(data):
    index=find_nearst_index_for_space(mod_data,i+100000)
    for word in mod_data[i:index].split():
        if len(word)>2 and check_bengali_word(word):
            modified_corpus+=word+""  
    i=index
##now for the last chunk of data    
for word in mod_data[i:].split():
    if len(word)>2 and check_bengali_word(word):
        modified_corpus+=word+""     

#### Saving the modified corpus

In [0]:
textfile = open('/home/rohan/CMI/SEM_3/Text_analytics/Corpus.txt', 'w')
textfile.write(modified_corpus)
textfile.close()